In [1]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k=None,
                      device=0)
classifier("I love this!")

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
Device set to use 0


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.00852868054062128},
  {'label': 'neutral', 'score': 0.005764597095549107},
  {'label': 'anger', 'score': 0.004419785924255848},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'disgust', 'score': 0.0016119946958497167},
  {'label': 'fear', 'score': 0.00041385178337804973}]]

In [4]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [5]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548429727554321},
  {'label': 'neutral', 'score': 0.16985104978084564},
  {'label': 'sadness', 'score': 0.11640841513872147},
  {'label': 'surprise', 'score': 0.020700562745332718},
  {'label': 'disgust', 'score': 0.019100630655884743},
  {'label': 'joy', 'score': 0.015161226503551006},
  {'label': 'anger', 'score': 0.003935141488909721}]]

In [9]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [10]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [11]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296030521392822},
 {'label': 'neutral', 'score': 0.14038534462451935},
 {'label': 'fear', 'score': 0.06816216558218002},
 {'label': 'joy', 'score': 0.04794243350625038},
 {'label': 'anger', 'score': 0.00915635284036398},
 {'label': 'disgust', 'score': 0.0026284728664904833},
 {'label': 'sadness', 'score': 0.00212215818464756}]

In [12]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [13]:
predictions[3]

[{'label': 'fear', 'score': 0.9281678199768066},
 {'label': 'anger', 'score': 0.03219113498926163},
 {'label': 'neutral', 'score': 0.012808755971491337},
 {'label': 'sadness', 'score': 0.008756887167692184},
 {'label': 'surprise', 'score': 0.00859794206917286},
 {'label': 'disgust', 'score': 0.008431871421635151},
 {'label': 'joy', 'score': 0.0010455826995894313}]

In [14]:
sorted(predictions[0], key = lambda x: x["label"])

[{'label': 'anger', 'score': 0.00915635284036398},
 {'label': 'disgust', 'score': 0.0026284728664904833},
 {'label': 'fear', 'score': 0.06816216558218002},
 {'label': 'joy', 'score': 0.04794243350625038},
 {'label': 'neutral', 'score': 0.14038534462451935},
 {'label': 'sadness', 'score': 0.00212215818464756},
 {'label': 'surprise', 'score': 0.7296030521392822}]

In [19]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "natural"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}


def calculate_max_emotion_scores(predictions):
    emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["score"], reverse=True)
        for index, label in enumerate(emotion_labels):
            emotion_scores[label].append(sorted_predictions[index]["score"])
    
    # Ensure no empty lists in emotion_scores
    return {label: np.max(scores) if scores else 0 for label, scores in emotion_scores.items()}

In [20]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [21]:
emotion_scores

{'anger': [0.9671575427055359,
  0.9425276517868042,
  0.9723208546638489,
  0.7326855063438416,
  0.8843896985054016,
  0.7271746397018433,
  0.8725658655166626,
  0.8202813267707825,
  0.9155242443084717,
  0.8603722453117371],
 'disgust': [0.27359145879745483,
  0.31934985518455505,
  0.11169011145830154,
  0.3514842987060547,
  0.2726130485534668,
  0.2719029188156128,
  0.3853569030761719,
  0.29216691851615906,
  0.2794812321662903,
  0.17792725563049316],
 'fear': [0.10908287018537521,
  0.1954360008239746,
  0.10400652140378952,
  0.15072286128997803,
  0.12224285304546356,
  0.11962231993675232,
  0.18529145419597626,
  0.23448744416236877,
  0.24911755323410034,
  0.10400652140378952],
 'joy': [0.09362712502479553,
  0.14334779977798462,
  0.07876545190811157,
  0.07967229187488556,
  0.09504341334104538,
  0.10239032655954361,
  0.08459071815013885,
  0.07876545190811157,
  0.1356140822172165,
  0.07876545190811157],
 'sadness': [0.06413349509239197,
  0.07713030278682709,
 

In [22]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "natural"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}


for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [35:35<00:00,  2.43it/s]  


In [23]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [24]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,natural,isbn13
0,0.967158,0.273591,0.109083,0.093627,0.064133,0.051363,0.040564,9780002005883
1,0.942528,0.319350,0.195436,0.143348,0.077130,0.051363,0.040564,9780002261982
2,0.972321,0.111690,0.104007,0.078765,0.064133,0.051363,0.040564,9780006178736
3,0.732686,0.351484,0.150723,0.079672,0.064133,0.051363,0.040564,9780006280897
4,0.884390,0.272613,0.122243,0.095043,0.064133,0.051363,0.040564,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.980877,0.305738,0.148208,0.127782,0.043363,0.030656,0.009569,9788172235222
5193,0.883199,0.338892,0.227765,0.078765,0.064133,0.051363,0.040564,9788173031014
5194,0.947779,0.339218,0.141733,0.066685,0.057625,0.009929,0.009055,9788179921623
5195,0.951104,0.368111,0.214133,0.078765,0.064133,0.051363,0.040564,9788185300535


In [25]:
books = pd.merge(books, emotions_df, on="isbn13")

In [26]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,natural
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.967158,0.273591,0.109083,0.093627,0.064133,0.051363,0.040564
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.942528,0.319350,0.195436,0.143348,0.077130,0.051363,0.040564
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.972321,0.111690,0.104007,0.078765,0.064133,0.051363,0.040564
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.732686,0.351484,0.150723,0.079672,0.064133,0.051363,0.040564
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.884390,0.272613,0.122243,0.095043,0.064133,0.051363,0.040564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.980877,0.305738,0.148208,0.127782,0.043363,0.030656,0.009569
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.883199,0.338892,0.227765,0.078765,0.064133,0.051363,0.040564
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.947779,0.339218,0.141733,0.066685,0.057625,0.009929,0.009055
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.951104,0.368111,0.214133,0.078765,0.064133,0.051363,0.040564


In [27]:
books.to_csv("books_with_emotions.csv", index=False)